In [3]:
import json
import os

def generate_lammps_files(json_path):
    # Load the JSON
    with open(json_path, "r") as f:
        data = json.load(f)

    element = data["tersoff"]["element1"]  # assuming element1 == element2 == element3

    # Create tersoff file content
    tersoff_header = f"""# Tersoff parameters

# these entries are in LAMMPS "metal" units:
#   A,B = eV; lambda1,lambda2,lambda3 = 1/Angstroms; R,D = Angstroms
#   other quantities are unitless

# format of a single entry (one or more lines):
#   element 1, element 2, element 3, 
#   m, gamma, lambda3, c, d, costheta0, n, beta, 
#   lambda2, B, R, D, lambda1, A

"""

    tersoff_params = data["tersoff"]
    line1 = f"{tersoff_params['element1']}  {tersoff_params['element2']}  {tersoff_params['element3']}  {tersoff_params['m']}  {tersoff_params['gamma']} {tersoff_params['lambda3']} {tersoff_params['c']} {tersoff_params['d']} {tersoff_params['h']} {tersoff_params['n']} {tersoff_params['beta']}\n"
    line2 = f"            {tersoff_params['lambda2']} {tersoff_params['B']} {tersoff_params['R']} {tersoff_params['D']} {tersoff_params['lambda1']} {tersoff_params['A']}\n"

    tersoff_content = tersoff_header + line1 + line2

    # Write tersoff file
    tersoff_filename = f"{element}.tersoff"
    with open(tersoff_filename, "w") as f:
        f.write(tersoff_content)

    # Create LAMMPS input snippet
    scaling = data["scaling"]
    lammps_input = f"""# ----- System -----
units metal
boundary p p p
box tilt large
atom_style atomic
read_data  your file here
replicate 1 1 1
variable T0 equal 300
variable P0 equal 1

# ----- Force Field -----
pair_style hybrid/overlay tersoff lj/cut/scaling 14
pair_coeff * * tersoff  {tersoff_filename} {element}
pair_coeff 1 1 lj/cut/scaling {scaling['epsilon']} {scaling['sigma']} {scaling['k1']} {scaling['k2']} {scaling['RcLR']}
neighbor 2.0 nsq
neigh_modify delay 0 every 1 check yes
"""

    # Write input snippet
    lammps_input_filename = f"{element}.in"
    with open(lammps_input_filename, "w") as f:
        f.write(lammps_input)

    print(f"✅ Generated: {tersoff_filename} and {lammps_input_filename}")

# === Example usage ===
generate_lammps_files("tersoff_files/Ag_hybop.json")  # Replace with your JSON filename


✅ Generated: Ag.tersoff and Ag.in
